## Scikit learn to ONNX
- Neuromophic 연구 : ONNX <-> ML (Scikit learn)
- 랜덤 포레스트 모델 (RandomForest)

In [6]:
# Iris Data Load 
# Train a model.
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(112, 4) (38, 4) (112,) (38,)


In [7]:
# Random Forest Classifier Fit

from sklearn.ensemble import RandomForestClassifier

# 모델 생성 및 학습
clr = RandomForestClassifier()
clr.fit(X_train, y_train)
clr

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [8]:
# Convert into ONNX format
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(clr, initial_types=initial_type)

# .onnx 모델 지정
with open("rf_iris.onnx", "wb") as f:
    f.write(onx.SerializeToString())


In [9]:
# Compute the prediction with ONNX Runtime
import onnxruntime as rt
import numpy as np

sess = rt.InferenceSession("rf_iris.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onx = sess.run([label_name], {input_name: X_test.astype(np.float32)})[0]

In [10]:
pred_onx

array([2, 0, 2, 2, 0, 1, 1, 0, 1, 2, 0, 1, 1, 0, 2, 2, 2, 2, 1, 0, 2, 0,
       1, 2, 0, 2, 2, 0, 1, 0, 1, 1, 1, 1, 1, 2, 0, 1], dtype=int64)

In [11]:
y_test

array([1, 0, 2, 2, 0, 1, 1, 0, 1, 2, 0, 1, 1, 0, 2, 2, 2, 2, 1, 0, 2, 0,
       1, 2, 0, 2, 2, 0, 1, 0, 1, 1, 1, 2, 1, 2, 0, 1])

In [12]:
# Accuracy 평가지표
from sklearn.metrics import accuracy_score

# Predict에 대한 Accuracy Score
print('accuracy_score', float(accuracy_score(y_test, pred_onx))) 

accuracy_score 0.9473684210526315
